# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [3]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [4]:
data = pd.read_csv("data/kddcup.data_10_percent", header=None, names = col_names)

In [5]:
data.shape

(494021, 42)

# 前処理
## カテゴリ化

In [6]:
data.label.value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [7]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [8]:
data.label2.value_counts()

atack      396743
normal.     97278
Name: label2, dtype: int64

In [9]:
data['label3'] = data.label.copy()

In [10]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop'),'label3'] = 'DoS'

In [11]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit'),'label3'] = 'U2R'

In [12]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster'),'label3'] = 'R2L'

In [13]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan'),'label3'] = 'Probe'

In [14]:
data.label3.value_counts()

DoS        391458
normal.     97278
Probe        4107
R2L          1126
U2R            52
Name: label3, dtype: int64

## サンプリング

In [15]:
data = resample(data,n_samples=40000,random_state=0)

In [16]:
data.shape

(40000, 44)

## 数値化

In [17]:
le_protocol_type = preprocessing.LabelEncoder()

In [18]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [19]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [20]:
le_service = preprocessing.LabelEncoder()

In [21]:
le_service.fit(data.service)

LabelEncoder()

In [22]:
data.service = le_service.transform(data.service)

In [23]:
le_flag = preprocessing.LabelEncoder()

In [24]:
le_flag.fit(data.flag)

LabelEncoder()

In [25]:
data.flag = le_flag.transform(data.flag)

In [26]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,40000.000000,40000.000000,40000.000000,40000.000000,4.000000e+04,4.000000e+04,40000.000000,40000.000000,40000.0,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,48.733500,0.464925,23.251775,6.845000,1.325639e+03,9.129235e+02,0.000050,0.006825,0.0,0.038700,...,232.128500,189.384550,0.756684,0.030395,0.603185,0.007170,0.173805,0.173673,0.059097,0.058300
std,744.715442,0.575177,13.404260,2.256735,4.593989e+04,3.573975e+04,0.007071,0.139208,0.0,0.825178,...,64.988061,105.707066,0.409373,0.108182,0.481151,0.046449,0.378104,0.378547,0.232462,0.231711
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,14.000000,8.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.0,0.000000,...,255.000000,55.000000,0.490000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,14.000000,8.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.0,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,40.000000,8.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.0,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,42448.000000,2.000000,62.000000,9.000000,5.133877e+06,5.151049e+06,1.000000,3.000000,0.0,30.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
data.shape

(40000, 44)

## ラベルの分離

In [28]:
y_train_1 = data.label.copy() 

In [29]:
y_train_2 = data.label2.copy()

In [30]:
y_train_3 = data.label3.copy()

In [31]:
x_train = data.drop(['label','label2','label3'],axis=1)

In [32]:
x_train.shape

(40000, 41)

In [33]:
y_train_1.shape

(40000,)

In [34]:
y_train_2.shape

(40000,)

In [35]:
y_train_3.shape

(40000,)

## 標準化

In [36]:
ss = preprocessing.StandardScaler()

In [37]:
ss.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [38]:
x_train = ss.transform(x_train)

In [39]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [40]:
pd.DataFrame(x_train,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,40000.0,4.000000e+04,...,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04
mean,-1.305939e-15,1.741329e-16,9.942214e-16,-1.733003e-16,4.426895e-16,2.264166e-16,-4.672501e-16,2.377213e-15,0.0,-1.465841e-18,...,-3.249051e-15,2.361167e-16,-2.998712e-17,-4.054701e-16,-1.426914e-16,2.856312e-16,9.592743e-16,1.480288e-16,3.075540e-15,-5.998313e-16
std,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,0.0,1.000013e+00,...,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00
min,-6.543991e-02,-8.083272e-01,-1.734677e+00,-3.033181e+00,-2.885630e-02,-2.554396e-02,-7.071245e-03,-4.902792e-02,0.0,-4.689954e-02,...,-3.556521e+00,-1.782160e+00,-1.848420e+00,-2.809670e-01,-1.253645e+00,-1.543744e-01,-4.596818e-01,-4.587928e-01,-2.542229e-01,-2.516081e-01
25%,-6.543991e-02,-8.083272e-01,-6.902202e-01,5.118078e-01,-2.787675e-02,-2.554396e-02,-7.071245e-03,-4.902792e-02,0.0,-4.689954e-02,...,3.519383e-01,-1.271308e+00,-6.514531e-01,-2.809670e-01,-1.253645e+00,-1.543744e-01,-4.596818e-01,-4.587928e-01,-2.542229e-01,-2.516081e-01
50%,-6.543991e-02,-8.083272e-01,-6.902202e-01,5.118078e-01,-1.753702e-02,-2.554396e-02,-7.071245e-03,-4.902792e-02,0.0,-4.689954e-02,...,3.519383e-01,6.207368e-01,5.943700e-01,-2.809670e-01,8.247318e-01,-1.543744e-01,-4.596818e-01,-4.587928e-01,-2.542229e-01,-2.516081e-01
75%,-6.543991e-02,9.302913e-01,1.249486e+00,5.118078e-01,-6.391887e-03,-2.554396e-02,-7.071245e-03,-4.902792e-02,0.0,-4.689954e-02,...,3.519383e-01,6.207368e-01,5.943700e-01,8.878418e-02,8.247318e-01,-1.543744e-01,-4.596818e-01,-4.587928e-01,-2.542229e-01,-2.516081e-01
max,5.693422e+01,2.668910e+00,2.890776e+00,9.549314e-01,1.117246e+02,1.441029e+02,1.414178e+02,2.150171e+01,0.0,3.630933e+01,...,3.519383e-01,6.207368e-01,5.943700e-01,8.962811e+00,8.247318e-01,2.137472e+01,2.185128e+00,2.182920e+00,4.047603e+00,4.064157e+00


## 学習

In [41]:
pca = PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


In [42]:
clf = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.17782794100389229,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [43]:
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [44]:
x_train3 = pca.transform(x_train)

In [45]:
t1=time.perf_counter()
clf.fit(x_train3,y_train_3)
t2=time.perf_counter()

In [46]:
print(t2-t1,"秒")

0.8217600180068985 秒


## 予測

In [47]:
t1=time.perf_counter()
pred=clf.predict(x_train3)
t2=time.perf_counter()

In [48]:
print(t2-t1,"秒")

0.543758029001765 秒


In [49]:
print(classification_report(y_train_3, pred))
print(confusion_matrix(y_train_3, pred))

             precision    recall  f1-score   support

        DoS       1.00      1.00      1.00     31669
      Probe       0.98      0.86      0.92       348
        R2L       0.89      0.41      0.56        98
        U2R       1.00      0.67      0.80         6
    normal.       0.98      1.00      0.99      7879

avg / total       0.99      0.99      0.99     40000

[[31586     6     0     0    77]
 [   20   299     0     0    29]
 [    0     0    40     0    58]
 [    1     0     0     4     1]
 [    7     0     5     0  7867]]
